In [83]:
%pip install torch torch-geometric
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [84]:
import pandas as pd

genuine_users = pd.read_csv('./cresci-2017/genuine_accounts/users.csv', encoding_errors='replace')
genuine_tweets = pd.read_csv('cresci-2017/genuine_accounts/tweets.csv', encoding_errors='replace')
social_spam_users_1 = pd.read_csv('cresci-2017/social_spambots_1/users.csv', encoding_errors='replace')
social_spam_tweets_1 = pd.read_csv('cresci-2017/social_spambots_1/tweets.csv', encoding_errors='replace')
social_spam_users_2 = pd.read_csv('cresci-2017/social_spambots_2/users.csv', encoding_errors='replace')
social_spam_tweets_2 = pd.read_csv('cresci-2017/social_spambots_2/tweets.csv', encoding_errors='replace')
social_spam_users_3 = pd.read_csv('cresci-2017/social_spambots_3/users.csv', encoding_errors='replace')
social_spam_tweets_3 = pd.read_csv('cresci-2017/social_spambots_3/tweets.csv', encoding_errors='replace')
traditional_spam_users = pd.read_csv('cresci-2017/traditional_spambots_1/users.csv', encoding_errors='replace')
traditional_spam_tweets = pd.read_csv('cresci-2017/traditional_spambots_1/tweets.csv', encoding_errors='replace')
# fake_follower_users = pd.read_csv('cresci-2017/fake_followers/users.csv', encoding_errors='replace')
# fake_follower_tweets = pd.read_csv('cresci-2017/fake_followers/tweets.csv', encoding_errors='replace')

# # example of getting only reply tweets
# filtered_genuine_tweeets = genuine_tweets[genuine_tweets['in_reply_to_user_id'] != 0.0]

print(genuine_users.columns)
print(genuine_tweets.columns)
print(social_spam_users_1.columns)
print(social_spam_tweets_1.columns)
print(social_spam_users_2.columns)
print(social_spam_tweets_2.columns)
print(social_spam_users_3.columns)
print(social_spam_tweets_3.columns)
print(traditional_spam_users.columns)
print(traditional_spam_tweets.columns)

/tmp/ipykernel_1677327/823921838.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  genuine_tweets = pd.read_csv('cresci-2017/genuine_accounts/tweets.csv', encoding_errors='replace')
/tmp/ipykernel_1677327/823921838.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  social_spam_tweets_1 = pd.read_csv('cresci-2017/social_spambots_1/tweets.csv', encoding_errors='replace')
/tmp/ipykernel_1677327/823921838.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  social_spam_tweets_2 = pd.read_csv('cresci-2017/social_spambots_2/tweets.csv', encoding_errors='replace')
/tmp/ipykernel_1677327/823921838.py:10: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  social_spam_tweets_3 = pd.read_csv('cresci-2017/social_spambots_3/tweets.csv', encoding_errors='replace')


Index(['id', 'name', 'screen_name', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'url', 'lang',
       'time_zone', 'location', 'default_profile', 'default_profile_image',
       'geo_enabled', 'profile_image_url', 'profile_banner_url',
       'profile_use_background_image', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https',
       'profile_sidebar_border_color', 'profile_background_tile',
       'profile_sidebar_fill_color', 'profile_background_image_url',
       'profile_background_color', 'profile_link_color', 'utc_offset',
       'is_translator', 'follow_request_sent', 'protected', 'verified',
       'notifications', 'description', 'contributors_enabled', 'following',
       'created_at', 'timestamp', 'crawled_at', 'updated', 'test_set_1',
       'test_set_2'],
      dtype='object')
Index(['id', 'text', 'source', 'user_id', 'truncated', 'in_reply_to_status_id',
       'in_reply_to_user_id

In [85]:
# Combining data into a single dataframe
genuine_users['label'] = 0
social_spam_users_1['label'] = 1
social_spam_users_2['label'] = 1
social_spam_users_3['label'] = 1
traditional_spam_users['label'] = 1

all_users = pd.concat([
    genuine_users,
    social_spam_users_1,
    social_spam_users_2,
    social_spam_users_3,
    traditional_spam_users
], ignore_index=True)

# all_users.head()

In [86]:
# Preparing relevant features
import numpy as np

features = [
    'statuses_count', 'followers_count', 'friends_count',
    'favourites_count', 'listed_count', 'default_profile',
    'default_profile_image', 'geo_enabled', 'protected', 'verified'
]

for col in features:
    if all_users[col].dtype == bool:
        all_users[col] = all_users[col].astype(int)

X = all_users[features].fillna(0).to_numpy(dtype=np.float32)
y = all_users['label'].to_numpy(dtype=np.int64)


In [87]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import numpy as np

# # Sample graph with 5 nodes
# edge_index = torch.tensor([[0, 1, 2, 3, 4, 1],
#                            [1, 0, 3, 2, 1, 4]], dtype=torch.long)  # directed

# # Node features: tweet frequency, URL presence, hashtag density, etc.
# x = torch.tensor([
#     [0.7, 1.0, 0.5, 0.8],  # user 0
#     [0.2, 0.0, 0.1, 0.4],  # user 1
#     [0.9, 1.0, 0.6, 0.7],  # user 2
#     [0.1, 0.0, 0.2, 0.3],  # user 3
#     [0.8, 1.0, 0.4, 0.9],  # user 4
# ], dtype=torch.float)

# # Labels: 1 = bot, 0 = human
# y = torch.tensor([1, 0, 1, 0, 1], dtype=torch.long)

# # Use a basic graph object
# data = Data(x=x, edge_index=edge_index, y=y)


In [ ]:
import pickle
import random

def save_rng(name="rng_states.pkl'"):
    
    python_rng_state = random.getstate()
    numpy_rng_state = np.random.get_state()
    
    torch_rng_state = torch.get_rng_state()

    # If using CUDA:
    if torch.cuda.is_available():
        torch_cuda_rng_state = torch.cuda.get_rng_state()
    else:
        torch_cuda_rng_state = None
    
    with open(name, 'wb') as f:
        pickle.dump({
            'python': python_rng_state,
            'numpy': numpy_rng_state,
            'torch': torch_rng_state,
            'cuda': torch_cuda_rng_state
        }, f)


def load_rng(name='rng_states.pkl'):
    with open(name, 'rb') as f:
        states = pickle.load(f)

        random.setstate(states['python'])
        np.random.set_state(states['numpy'])
        torch.set_rng_state(states['torch'])

        if torch.cuda.is_available() and states['cuda'] is not None:
            torch.cuda.set_rng_state(states['cuda'])

In [ ]:
# save_rng()
# load_rng()

GNN model


In [90]:
from torch.nn import Linear

class BotClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.lin(x)
        return x


In [ ]:
from sklearn.neighbors import kneighbors_graph

x_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)

adj = kneighbors_graph(X, n_neighbors=5, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj.nonzero()), dtype=torch.long)


In [132]:
from sklearn.model_selection import train_test_split

idx = np.arange(len(X))
train_idx, test_idx = train_test_split(idx, test_size=0.2, stratify=y)

# Create train/test masks
train_mask = torch.zeros(len(X), dtype=torch.bool)
test_mask = torch.zeros(len(X), dtype=torch.bool)
train_mask[train_idx] = True
test_mask[test_idx] = True


data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor,
            train_mask=train_mask, test_mask=test_mask)
model = BotClassifier(input_dim=x_tensor.shape[1], hidden_dim=32, output_dim=2)

Reinfrocement learning

In [ ]:
input_dim = X.shape[1]  # which is 10
model = BotClassifier(input_dim=input_dim, hidden_dim=8, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def compute_reward(pred, y_true):
    return (pred.argmax(dim=1) == y_true).float() * 1.0 - 0.5  # +1 or -0.5

for epoch in range(1000):
    model.train()
    optimizer.zero_grad()

    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])

    # Simulate "reward" from environment
    reward = compute_reward(out[data.train_mask], data.y[data.train_mask]).mean()

    # RL-inspired update: reward modulates loss
    total_loss = loss - 0.01 * reward  # encourage higher reward
    total_loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        acc = (out[data.test_mask].argmax(dim=1) == data.y[data.test_mask]).float().mean().item()
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Acc: {acc:.2f}, Reward: {reward.item():.2f}")



Epoch 0, Loss: 270.2231, Acc: 0.37, Reward: -0.13
Epoch 10, Loss: 26.2541, Acc: 0.44, Reward: -0.05
Epoch 20, Loss: 8.4449, Acc: 0.58, Reward: 0.08
Epoch 30, Loss: 2.1896, Acc: 0.90, Reward: 0.40
Epoch 40, Loss: 1.5877, Acc: 0.88, Reward: 0.38
Epoch 50, Loss: 0.9615, Acc: 0.79, Reward: 0.30
Epoch 60, Loss: 0.8145, Acc: 0.78, Reward: 0.29
Epoch 70, Loss: 0.6736, Acc: 0.74, Reward: 0.25
Epoch 80, Loss: 0.5730, Acc: 0.77, Reward: 0.28
Epoch 90, Loss: 0.5403, Acc: 0.78, Reward: 0.28


In [135]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

model.eval()
preds = model(data).argmax(dim=1)
print("Predicted:", preds.tolist())
print("Actual:   ", data.y.tolist())


y_pred = preds.cpu().numpy()
y_true = data.y.cpu().numpy()

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)

print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")



Predicted: [1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1

In [ ]:
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict()
# }, 'model.pth')

In [ ]:
feature_df = pd.DataFrame(X, columns=features)
feature_df['true_label'] = y
feature_df['predicted'] = y_pred

false_positives = feature_df[(feature_df["true_label"] == 0) & (feature_df["predicted"] == 1)]
false_negatives = feature_df[(feature_df["true_label"] == 1) & (feature_df["predicted"] == 0)]
true_positives = feature_df[(feature_df["true_label"] == 1) & (feature_df["predicted"] == 1)]
true_negatives = feature_df[(feature_df["true_label"] == 0) & (feature_df["predicted"] == 0)]
bots = feature_df[feature_df["true_label"] == 1]
humans = feature_df[feature_df["true_label"] == 0]
predict_bot = feature_df[feature_df["predicted"] == 1]
predict_human = feature_df[feature_df["predicted"] == 0]



0.41689373297002724
0.1459620711698274


In [112]:
# print(sum(false_positives["default_profile"])/len(false_positives))
# print(sum(feature_df["default_profile"])/len(all_users))

print("\nAll Users:")
print(all_users.describe())

print("\nActual Bots:")
print(bots.describe())
print("\nActual Humans:")
print(humans.describe())

print("\nPredicted Bot:")
print(predict_bot.describe())
print("\nPredicted Human:")
print(predict_human.describe())


print("\nTrue Positives (Predicted Bot, Actually Bot):")
print(true_positives.describe())
print("\nTrue Negatives (Predicted Human, Actually Human):")
print(true_negatives.describe())

print("\nFalse Positives (Predicted Bot, Actually Bot):")
print(false_positives.describe())
print("\nFalse Negatives (Predicted Human, Actually Human):")
print(false_negatives.describe())



All Users:
                 id  statuses_count  followers_count  friends_count  \
count  9.386000e+03     9386.000000      9386.000000    9386.000000   
mean   1.322960e+09     7085.424782       903.678564     684.365864   
std    1.045865e+09    20700.901052     11673.778672    1974.945285   
min    6.780330e+05        1.000000         0.000000       0.000000   
25%    2.645520e+08       52.000000         5.000000      40.000000   
50%    1.245040e+09      115.000000        91.000000     161.000000   
75%    2.360648e+09     5346.750000       530.750000     656.000000   
max    3.164942e+09   399555.000000    986837.000000   90861.000000   

       favourites_count  listed_count  default_profile  default_profile_image  \
count       9386.000000   9386.000000           1370.0                   60.0   
mean        1749.888131      9.210739              1.0                    1.0   
std         7397.236025    103.845299              0.0                    0.0   
min            0.000000 

In [108]:
from sklearn.metrics import accuracy_score
import copy

# Baseline accuracy with original data
model.eval()
baseline_preds = model(data).argmax(dim=1)
baseline_acc = accuracy_score(data.y.cpu().numpy(), baseline_preds.cpu().numpy())
print(f"\nBaseline Accuracy: {baseline_acc:.4f}")

# Compute permutation importance
importances = []
X_original = data.x.clone()

for i, feat in enumerate(features):
    X_permuted = X_original.clone()
    X_permuted[:, i] = X_permuted[torch.randperm(X_permuted.size(0)), i]  # shuffle feature column
    
    data_permuted = copy.deepcopy(data)
    data_permuted.x = X_permuted

    with torch.no_grad():
        permuted_preds = model(data_permuted).argmax(dim=1)
        permuted_acc = accuracy_score(data.y.cpu().numpy(), permuted_preds.cpu().numpy())

    drop = baseline_acc - permuted_acc
    importances.append(drop)
    print(f"Feature: {feat:25s} | Accuracy Drop: {drop:.4f}")

# Display sorted importance
importance_df = pd.DataFrame({
    'feature': features,
    'accuracy_drop': importances
}).sort_values(by='accuracy_drop', ascending=False)

print("\nSorted Feature Importance:")
print(importance_df)



Baseline Accuracy: 0.9436
Feature: statuses_count            | Accuracy Drop: 0.0424
Feature: followers_count           | Accuracy Drop: 0.0745
Feature: friends_count             | Accuracy Drop: 0.0347
Feature: favourites_count          | Accuracy Drop: 0.4450
Feature: listed_count              | Accuracy Drop: 0.0018
Feature: default_profile           | Accuracy Drop: 0.0000
Feature: default_profile_image     | Accuracy Drop: 0.0000
Feature: geo_enabled               | Accuracy Drop: 0.0002
Feature: protected                 | Accuracy Drop: 0.0000
Feature: verified                  | Accuracy Drop: 0.0000

Sorted Feature Importance:
                 feature  accuracy_drop
3       favourites_count       0.445025
1        followers_count       0.074473
0         statuses_count       0.042404
2          friends_count       0.034733
4           listed_count       0.001811
7            geo_enabled       0.000213
5        default_profile       0.000000
6  default_profile_image       0.00

In [124]:
# 
len(bots[bots["favourites_count"]>100]),len(true_positives[true_positives["favourites_count"]>100])
len(bots[bots["favourites_count"]>1000]),len(true_positives[true_positives["favourites_count"]>1000])

(17, 4)